# Spatial Statistics with Voyager

Based on the following tutorials:
* https://pachterlab.github.io/voyager/articles/visium_10x.html
* https://pachterlab.github.io/voyager/articles/vig1_visium_basic.html
* https://pachterlab.github.io/voyager/articles/vig2_visium.html
* https://pachterlab.github.io/voyager/articles/visium_10x_spatial.html

In [ ]:
library(dplyr)
library(Voyager)
library(SpatialExperiment)
library(SpatialFeatureExperiment)
library(SingleCellExperiment)
library(ggplot2)
library(scater)
library(rlang)
library(scuttle)
library(terra)
library(sf)
library(rmapshaper)
# library(scran)
library(stringr)
library(EBImage)
library(patchwork)
library(bluster)
library(rjson)
theme_set(theme_bw())

In [ ]:
# Layout
custom_theme <- function() {
  theme_bw() +
    theme(
      legend.text = element_text(size = 14),
      legend.title = element_text(size = 16, face = "bold"),
      axis.text = element_text(size = 12),
      axis.title = element_text(size = 14, face = "bold"),
      legend.position = "right",
      legend.box.just = "right"
    )
}
options(repr.plot.width = 20, repr.plot.height = 16)

In [ ]:
data_dir <- R.utils::getAbsolutePath('../../data')
mouse_dir <- glue::glue("{data_dir}/Visium_Mouse_Olfactory_Bulb/outs")

## Visium Files

### Scale Factors

The scalefactors_json.json file contains image metadata:
* **tissue_hires_scalef** and **tissue_lowres_scalef** are the ratio of the size of the high resolution (but not full resolution) and low resolution H&E image to the full resolution image.
* **fiducial_diameter_fullres** is the diameter of each fiducial spot used to align the spots to the H&E image in pixels in the full resolution image.
* **spot_diameter_fullres** is the diameter of each Visium spot in the full resolution H&E image in pixels. 

In [ ]:
fromJSON(file = glue::glue("{mouse_dir}/spatial/scalefactors_json.json"))

### Tissue Metadata

The tissue_positions_list.csv file contains information about each spot/barcode:
* **in_tissue** indicates whether each spot is in tissue (in_tissue, 1 means yes and 0 means no) as automatically detected by 
Space Ranger or manually annotated in the Loupe browser.
* **array_row** and **array_col** are the coordinates on the matrix of spots,
* **pxl_row_in_fullres** and **pxl_col_in_fullres** are the coordinates of the spots in the full resolution 
image.

In [ ]:
head(read.csv(glue::glue("{mouse_dir}/spatial/tissue_positions.csv")))

# Read Visium Data

In [ ]:
#Original way to load Visium Data
#raw_sfe <- SpatialFeatureExperiment::read10xVisiumSFE(dirs = mouse_dir, samples = ".", type = "sparse", data = "raw")
#Voyager::plotImage(raw_sfe)
#transposed_raw_sfe <- SpatialFeatureExperiment::transpose(raw_sfe)

In [ ]:
# Read pre-processed file
raw_sfe <- readRDS(glue::glue("{data_dir}/Visium_Mouse_Olfactory_Bulb.rds"))
Voyager::plotImage(raw_sfe)
transposed_raw_sfe <- raw_sfe

# Perform QC

In [ ]:
is_mt <- str_detect(rowData(transposed_raw_sfe)$symbol, "^mt-")
sum(is_mt)

In [ ]:
qc_sfe <- scuttle::addPerCellQCMetrics(transposed_raw_sfe, subsets = list(mito = is_mt))

In [ ]:
(scater::plotColData(qc_sfe, "sum", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(scater::plotColData(qc_sfe, "detected", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(scater::plotColData(qc_sfe, "subsets_mito_percent", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(patchwork::plot_layout(ncol=3, guides = "collect"))

In [ ]:
scater::plotColData(qc_sfe, x = "sum", y = "subsets_mito_percent", bins = 100) + custom_theme()

In [ ]:
# Normally MITO % is set to 20 - see what effect this has compared to 30.
processed_sfe <- qc_sfe[, qc_sfe$subsets_mito_percent < 20]
processed_sfe <- processed_sfe[rowSums(counts(processed_sfe)) > 0,]

In [ ]:
(plotColData(processed_sfe, "sum", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(plotColData(processed_sfe, "detected", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(plotColData(processed_sfe, "subsets_mito_percent", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(plot_layout(ncol=3, guides = "collect"))

In [ ]:
plotColData(processed_sfe, x = "sum", y = "subsets_mito_percent", bins = 100) + custom_theme()

# Before and after QC by Percentage

In [ ]:
(Voyager::plotSpatialFeature(qc_sfe, c("sum"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme() +
Voyager::plotSpatialFeature(qc_sfe, c("detected"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme() +
Voyager::plotSpatialFeature(qc_sfe, c("subsets_mito_percent"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme()) +
(plot_layout(ncol = 2, guides = "collect"))

In [ ]:
colData(processed_sfe)$nCounts <- colSums(counts(processed_sfe))

In [ ]:
(Voyager::plotSpatialFeature(processed_sfe, c("sum"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme() +
Voyager::plotSpatialFeature(processed_sfe, c("detected"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme() +
Voyager::plotSpatialFeature(processed_sfe, c("subsets_mito_percent"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme()) +
(plot_layout(ncol = 2, guides = "collect"))

# Plotting Metrics in Space

## Counts Per Spot in and out of Tissue

Plot the total unique molecular identifier (UMI) counts per spot.

In [ ]:
violin <- plotColData(processed_sfe, "nCounts", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top") + custom_theme()
spatial <- plotSpatialFeature(processed_sfe, "nCounts", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary", 
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) + custom_theme()
violin + spatial

In [ ]:
colData(processed_sfe)$nGenes <- colSums(counts(processed_sfe) > 0)

In [ ]:
violin <- plotColData(processed_sfe, "nGenes", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top") + custom_theme()
spatial <- plotSpatialFeature(processed_sfe, "nGenes", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary",
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) + custom_theme()
violin + spatial

In [ ]:
plotColData(processed_sfe, x = "nCounts", y = "nGenes", colour_by = "in_tissue") + custom_theme()

In [ ]:
mito_ind <- str_detect(rowData(processed_sfe)$symbol, "^mt-")
colData(processed_sfe)$prop_mito <- colSums(counts(processed_sfe)[mito_ind,]) / colData(processed_sfe)$nCounts

In [ ]:
violin <- plotColData(processed_sfe, "prop_mito", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top") + custom_theme()
spatial <- plotSpatialFeature(processed_sfe, "prop_mito", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary",
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) + custom_theme()
violin + spatial

# Only use in_tissue spots

In [ ]:
sfe_tissue <- processed_sfe[, colData(processed_sfe)$in_tissue]
sfe_tissue <- sfe_tissue[rowSums(counts(sfe_tissue)) > 0,]

In [ ]:
rowData(sfe_tissue)$means <- rowMeans(counts(sfe_tissue))
rowData(sfe_tissue)$vars <- rowVars(counts(sfe_tissue))
# Coefficient of variance
rowData(sfe_tissue)$cv2 <- rowData(sfe_tissue)$vars/rowData(sfe_tissue)$means^2

In [ ]:
plotRowData(sfe_tissue, x = "means", y = "vars", bins = 50) +
    geom_abline(slope = 1, intercept = 0, color = "red") +
    scale_x_log10() + scale_y_log10() +
    scale_fill_distiller(palette = "Blues", direction = 1) +
    annotation_logticks() +
    coord_equal() + custom_theme()